# N electrons meet in a crystal

<center><img src="./images/qdef-banner.png" style="width: 500px"><br>
In this notebook there's a single interactive app that allows exploring the wavefunctions generated from different numbers of electrons in the t_2 and e orbitals for the O group.
    
</center>

In [1]:
from qdef import *
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import interact_manual, Layout, interactive

Reloading /Users/juan/Zia Lab/Codebase/qdef/data/CPGs.pkl ...


In [4]:
def config_printer(n1, n2, print_all):
    configargs = ((sp.Symbol('E'),n1), (sp.Symbol('T_2'),n2))
    config_latex = sp.Symbol('e')**n1 * sp.Symbol('t_2')**n2
    print("Calculating wave functions...")
    if configargs in config_printer.configs:
        econfig = config_printer.configs[configargs]
    else:
        econfig = CrystalElectronsLLCoupling('O',*configargs)
        config_printer.configs[configargs] = econfig
    terms = {}
    collected = {}
    for k,v in econfig.equiv_waves.items():
        if k.terms[-1] not in terms:
            terms[k.terms[-1]] = 0
            collected[k.terms[-1]] = []
        terms[k.terms[-1]] += 1
        collected[k.terms[-1]].append((k,v))

    terms = OrderedDict()
    econfig_maker = lambda x,y: sp.Symbol(sp.latex(x).lower()) * sp.Symbol(sp.latex(y).lower())
    for wave_key, wave in econfig.equiv_waves.items():
        this_term = wave_key.terms[-1]
        if this_term not in terms:
            terms[this_term] = []
        terms[this_term].append((wave_key,wave))
        # display(wave.as_ket())
    sorted_terms = sorted(terms.keys(), key= lambda x: x[0])
    for term in sorted_terms:
        terms[term] = list(sorted(terms[term], key=lambda x: str(x[0].γ)))
    counter = 0
    for term in sorted_terms:
        display(HTML('<hr>'))
        term_symb = sp.Symbol('{}^{%d}%s' % (term[0]*2+1, sp.latex(term[1])))
        display(term_symb)
        num_waves = len(terms[term])
        for wave_key, wave in terms[term]:
            e_config = sp.latex(reduce(sp.Mul,wave_key.electrons)).lower()
            multiplicity = int(wave_key.S*2+1)
            M = sp.latex(wave_key.M)
            γ = sp.latex(wave_key.γ)
            the_wave = sp.latex(as_det_ket(wave))
            wave_line = '\psi_{%d}(%s,%s,M=%s,%s) = %s' % (counter+1, e_config, term_symb, M, γ, the_wave)
            counter += 1
            display(Math(wave_line))
            if not print_all:
                if num_waves != 1:
                    plurality = 's' if (num_waves-1)>1 else ''
                    msg = '(... and %d other wave function%s ...)' % (num_waves-1, plurality)
                    display(Math('\\textrm{%s}' % msg))
                break
    else:
        display(HTML('<hr>'))
config_printer.configs = {}

config_box = widgets.HTMLMath('$\\Huge et_2^2$',align='center')
display(config_box)

def handle_slider_change(_):
    e_value = e_slider.value
    t2_value = t2_slider.value
    if e_value == 0 and t2_value == 0:
        config_box.value = '--'
    elif e_value == 0 and t2_value != 0:
        config_box.value = '$\\Huge  t_2^%d$' % t2_value
    elif e_value != 0 and t2_value == 0:
        config_box.value = '$\\Huge  e^%d$' % e_value
    elif e_value == 1 and t2_value == 1:
        config_box.value = '$\\Huge  e t_2$'
    elif e_value == 1 and t2_value != 1:
        config_box.value = '$\\Huge  e t_2^%d$' % (t2_value)
    elif e_value != 1 and t2_value == 1:
        config_box.value = '$\\Huge  e^%d t_2$' % (e_value)
    else:
        config_box.value = '$\\Huge  e^%d t_2^%d$' % (e_value, t2_value)

e_slider = widgets.IntSlider(value=1,
                           min=0, max=4,
                           step=1, description = 'e')
t2_slider = widgets.IntSlider(value=2,
                           min=0, max=6,
                           step=1, description = 't_2')

e_slider.observe(handle_slider_change)
t2_slider.observe(handle_slider_change)

im = interact_manual(config_printer,
                n1 = e_slider,
                n2 = t2_slider,
                print_all = widgets.Checkbox(value=False));
im.widget.children[-2].description = 'Calculate Wave Functions'
im.widget.children[-2].layout = Layout(width='300px', height='40px')


HTMLMath(value='$\\Huge et_2^2$')

interactive(children=(IntSlider(value=1, description='e', max=4), IntSlider(value=2, description='t_2', max=6)…